# Heart disease 
Model to predict occurence of heart failure/attack

**Model type: Decision tree**
 

In [2]:
import numpy as np
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report


In [3]:
health_data = pd.read_csv('heart_disease.csv')

health_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,Heart disease
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [4]:
from sklearn.model_selection import train_test_split
model = DecisionTreeClassifier()

x = health_data.drop(columns=['Heart disease'])
y = health_data['Heart disease']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

model.fit(x_train, y_train)
heart_disease_prediction = model.predict(x_test)
detection_accuracy = model.score(x_test, y_test)

detection_accuracy

0.8032786885245902

In [5]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()

rf_model.fit(x_train, y_train)
rf_predict = rf_model.predict(x_test)

rf_acc = rf_model.score(x_test, y_test)

rf_acc 

0.8524590163934426

# Improved model(Added Features)

In [6]:
import xgboost as xgb 
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

heart_data = pd.read_csv('heart.csv')
heart_data = heart_data.dropna()
heart_data

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [7]:
data = pd.DataFrame({'RestingECG': heart_data['RestingECG'], 'ChestPainType': heart_data['ChestPainType']})

In [8]:
heart_data['Sex'] = (heart_data['Sex'] == 'M').astype(int)
heart_data['ExerciseAngina'] = (heart_data['ExerciseAngina'] == 'Y').astype(int)
heart_data['ST_Slope'] = (heart_data['ST_Slope'] == 'Up').astype(int)

encoder = OneHotEncoder(handle_unknown='ignore')
encoded = encoder.fit_transform(data[['RestingECG', 'ChestPainType']]).toarray()

heart_data.update(encoded)

heart_data


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,ATA,140,289,0,Normal,172,0,0.0,1,0
1,49,0,NAP,160,180,0,Normal,156,0,1.0,0,1
2,37,1,ATA,130,283,0,ST,98,0,0.0,1,0
3,48,0,ASY,138,214,0,Normal,108,1,1.5,0,1
4,54,1,NAP,150,195,0,Normal,122,0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,1,TA,110,264,0,Normal,132,0,1.2,0,1
914,68,1,ASY,144,193,1,Normal,141,0,3.4,0,1
915,57,1,ASY,130,131,0,Normal,115,1,1.2,0,1
916,57,0,ATA,130,236,0,LVH,174,0,0.0,0,1


In [9]:
nightly_model = RandomForestClassifier()
nightly_model.fit(x_train, y_train)

heart_failure_predict = nightly_model.predict(x_test)

beta_score = nightly_model.score(x_test, y_test)
beta_score

0.8524590163934426

In [10]:
metrics_report = classification_report(y_test, heart_failure_predict)

print(metrics_report)

              precision    recall  f1-score   support

           0       0.85      0.81      0.83        27
           1       0.86      0.88      0.87        34

    accuracy                           0.85        61
   macro avg       0.85      0.85      0.85        61
weighted avg       0.85      0.85      0.85        61

